<a href="https://colab.research.google.com/github/sidh26/NLP-assignments/blob/master/classwork/Polarity_Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
import numpy as np

In [5]:
from nltk import word_tokenize
from nltk.tokenize import MWETokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [6]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
twitter = pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv", error_bad_lines=False)
review = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz", lines=True)

In [25]:
data.shape

(5113, 5)

In [0]:
doc = twitter['TweetText'][0]

In [34]:
doc

'Now all @Apple has to do is get swype on the iphone and it will be crack. Iphone that is'

In [35]:
TextBlob(doc).sentiment.polarity

0.0

In [0]:
def textblob_polarity(doc):
    # print(doc[0])
    return TextBlob(doc[0]).sentiment.polarity

In [0]:
twitter['polarity'] = twitter[['TweetText']][:1000].apply(textblob_polarity, axis=1)
review['polarity'] = review[['reviewText']][:1000].apply(textblob_polarity, axis=1)

In [55]:
twitter['polarity'].head(10)

0    0.000000
1    0.500000
2    0.393750
3    0.541667
4    0.200000
5   -0.375000
6    0.362500
7    0.400000
8    1.000000
9    0.800000
Name: polarity, dtype: float64

In [60]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [0]:
def nltk_polarity(doc):
    # print(doc[0])
    return analyzer.polarity_scores(doc[0])

In [0]:
twitter['polarity'] = twitter[['TweetText']][:1000].apply(nltk_polarity, axis=1)

In [69]:
twitter['polarity'].head(10)

0    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1    {'neg': 0.0, 'neu': 0.801, 'pos': 0.199, 'comp...
2    {'neg': 0.0, 'neu': 0.591, 'pos': 0.409, 'comp...
3    {'neg': 0.0, 'neu': 0.814, 'pos': 0.186, 'comp...
4    {'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'comp...
5    {'neg': 0.172, 'neu': 0.828, 'pos': 0.0, 'comp...
6    {'neg': 0.0, 'neu': 0.788, 'pos': 0.212, 'comp...
7    {'neg': 0.0, 'neu': 0.695, 'pos': 0.305, 'comp...
8    {'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...
9    {'neg': 0.0, 'neu': 0.612, 'pos': 0.388, 'comp...
Name: polarity, dtype: object

In [0]:
analyzer = SentimentIntensityAnalyzer()
def nltk_senti(doc):
    d = analyzer.polarity_scores(doc[0])
    # using comppund instead of finding max of pos, neg and neu
    # using threshold values based on the github readme https://github.com/cjhutto/vaderSentiment
    if d['compound']>=0.05: 
        return 'positive'
    elif d['compound']<=-0.05:
        return 'negative'
    else:
        return 'neutral'

In [78]:
set(twitter['Sentiment'])

{'irrelevant', 'negative', 'neutral', 'positive'}

In [0]:
twitter['nltk_sentiment'] = twitter[['TweetText']].apply(nltk_senti, axis=1)

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [84]:
twitter['nltk_sentiment']

0        neutral
1       positive
2       positive
3       positive
4       positive
          ...   
5108     neutral
5109    positive
5110    negative
5111     neutral
5112     neutral
Name: nltk_sentiment, Length: 5113, dtype: object

In [87]:
confusion_matrix(twitter['Sentiment'],twitter['nltk_sentiment'])

array([[   0,  238, 1189,  262],
       [   0,  291,  138,  143],
       [   0,  302, 1234,  797],
       [   0,   47,   99,  373]])

In [89]:
accuracy_score(twitter['Sentiment'],twitter['nltk_sentiment'])

0.37121063954625466

# Glove part

In [0]:
!pip install glove_python
from glove import Corpus, Glove
import nltk
nltk.download('punkt')

     |████████████████████████████████| 266kB 2.8MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700331 sha256=e57b61e064c8273c0cbdc921a29f494a16ac2890a5d49e3a217edc8052ccbea2
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
Tweets = twitter["TweetText"].apply(nltk.word_tokenize)
corpus = Corpus()
corpus.fit(Tweets,window=10)
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.most_similar('good')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


[('great', 0.9819575722217783),
 ('or', 0.9799883887315579),
 ('nice', 0.9780000725086175),
 ('got', 0.9658829912285166)]